<a href="https://colab.research.google.com/github/hanzhang-store/Deep-learning/blob/main/Example1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
import cvxpy as cp
import numpy as np
import math
from scipy.linalg import block_diag

In [2]:
nx = 1;       # input dimension
nl = 4;       # number of layers
nk = 4;       # number of Neurons per layer (assume identical)
N  = nl*nk;   # Total number of neurons
nf = 1;       # output dimension

# Reduced order network
ml = 3;       # number of layers
mk = 3;       # number of neurons per layer (assume identical)
M  = ml*mk;   # Total number of neurons

In [3]:
vec = np.linspace(1,nk+1,nk)/nk
cos_vec = np.cos(vec*2*math.pi)
cos_vec = np.reshape(cos_vec,(4,1))
sin_vec = np.sin(vec*2*math.pi)
sin_vec = np.reshape(sin_vec,(4,1))
cos_vec_sq = np.matmul(cos_vec, sin_vec.T)

In [4]:
W0 = cos_vec
b0 = 0*sin_vec

In [5]:
W = []
b = []
if nl > 1 :
  for i in range(1,nl):
    W.append(((i+1))*cos_vec_sq)
    b.append((1/(i+1))*sin_vec)

In [6]:
Wl = sin_vec
bl = 0*np.ones((nf,1))
bm = b

Wcheck = block_diag(W[0],W[1],W[2])
a = np.hstack((Wcheck,np.zeros((N-nk,nk))))
W_ck = np.vstack((np.zeros((nk,N)),a))
W0_ck = np.vstack((W0,np.zeros((N-nk,nx))))
b_ck = np.vstack((b0,bm[0],bm[1],bm[2]))
Wl_ck = np.hstack((np.zeros((nf,N-nk)),Wl.T))
bl_ck = bl
nl = nl
nk = nk

In [7]:
# define class
class BaselineModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, num_classes):
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1) 
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3) 
        self.fc4 = nn.Linear(hidden_size3, hidden_size4) 
        self.fc5 = nn.Linear(hidden_size4, num_classes) 
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.relu(out)
        out = self.fc4(out)
        out = F.relu(out)
        out = self.fc5(out) #output layer is linear
        return out

In [8]:
input = torch.tensor([1.0], requires_grad=True)
output = torch.tensor([2.0], requires_grad=True)
model = BaselineModel(1,4,4,4,4,1) #construct NN for a single hidden layer network of dimension 10

loss_function = nn.L1Loss()
optimiser = optim.Adam(model.parameters())

In [9]:
# the epoch loop
for epoch in range(1000):
    running_loss = 0.0
    # zero the parameter gradients
    optimiser.zero_grad() 
    expect_output = model(input)
    loss = loss_function(output, expect_output) #loss
    loss.backward() 
    optimiser.step() #update weight and bias
    running_loss += loss.item()

In [10]:
model(input)

tensor([1.9999], grad_fn=<AddBackward0>)

In [11]:
for parameters in model.parameters():
    print(parameters)

Parameter containing:
tensor([[ 0.5097],
        [-0.0470],
        [ 0.9895],
        [ 0.3279]], requires_grad=True)
Parameter containing:
tensor([ 0.3850, -0.2148,  1.0761,  0.7789], requires_grad=True)
Parameter containing:
tensor([[ 0.2551,  0.3812,  0.1419, -0.2294],
        [ 0.5251, -0.4184,  0.6381,  0.5378],
        [-0.2641,  0.1251, -0.3192, -0.3494],
        [ 0.0957, -0.1294, -0.1515, -0.0461]], requires_grad=True)
Parameter containing:
tensor([-0.3040,  0.6130,  0.3636, -0.3127], requires_grad=True)
Parameter containing:
tensor([[ 0.3999,  0.1073, -0.1983,  0.3277],
        [ 0.0523,  0.6871, -0.4875,  0.4814],
        [-0.4410, -0.2998, -0.2008, -0.1327],
        [-0.1230, -0.0650,  0.4568, -0.1084]], requires_grad=True)
Parameter containing:
tensor([-0.3628,  0.3989,  0.3549,  0.0632], requires_grad=True)
Parameter containing:
tensor([[-3.9592e-01, -3.6099e-01,  9.5120e-02,  4.5206e-01],
        [ 2.0364e-01,  3.5436e-04, -2.8728e-01, -1.1203e-01],
        [-4.2381e-02

In [12]:
input = input.detach().numpy() #only run for initial, twice will be error
w0 = np.array(model.state_dict()['fc1.weight'])
b0 = np.array(model.state_dict()['fc1.bias'])
w1 = np.array(model.state_dict()['fc2.weight'])
b1 = np.array(model.state_dict()['fc2.bias'])
w2 = np.array(model.state_dict()['fc3.weight'])
b2 = np.array(model.state_dict()['fc3.bias'])
w3 = np.array(model.state_dict()['fc4.weight'])
b3 = np.array(model.state_dict()['fc4.bias'])
w4 = np.array(model.state_dict()['fc5.weight'])
b4 = np.array(model.state_dict()['fc5.bias'])

In [13]:
nx = 1;       # input dimension
nl = 4;       # number of layers
nk = 4;       # number of Neurons per layer (assume identical)
N  = nl*nk;   # Total number of neurons
nf = 1;       # output dimension

# Reduced order network
ml = 3;       # number of layers
mk = 3;       # number of neurons per layer (assume identical)
M  = ml*mk;   # Total number of neurons

In [14]:
Wcheck = block_diag(w1,w2,w3)
a = np.hstack((Wcheck,np.zeros((N-nk,nk))))
W_ck = np.vstack((np.zeros((nk,N)),a))
W0_ck = np.vstack((w0,np.zeros((N-nk,nx))))
b_ck = np.hstack((b0,b1,b2,b3)) # b的注意
b_ck = np.reshape(b_ck,(16,1))
Wl_ck = np.hstack((np.zeros((nf,N-nk)),w4))
bl_ck = b4

In [15]:
W = W_ck;
W0 = W0_ck;
Wf = Wl_ck;
b = b_ck;
bl = bl_ck;

nf = Wf.shape[0]
nx = W0.shape[1]

N = nl*nk;
M = ml*mk;


In [16]:
# J matrix
a = np.eye(mk)
bb = np.zeros((mk,nk-mk))
Jmn = np.hstack((a,bb))
J1a = np.kron((np.eye(ml)),Jmn)
J1b = np.zeros((M,(nl-ml)*nk))
J1 = np.hstack((J1a,J1b))
J1 = J1.T
J2 = np.ones((M,1))

In [17]:
T0 = cp.Variable((N,1))
T0 = cp.diag(T0)  # 

T0_r = cp.Variable((M,1))
T0_r = cp.diag(T0_r)  #T0_r also needs to be nonsingular

Tp = cp.Variable((N,1),nonneg=True) # 
Tp_r = cp.Variable((M,1),nonneg=True)

Tx = cp.Variable((N,M),nonneg=True)
Tc_p = cp.Variable((N,1),nonneg=True)

#Tx_r is substitute of T0_r * J1.T
#Tcp_r is substitute of T0_r * J2

In [18]:
sF = []
for i in range(1,ml):
  sF.append(cp.Variable((mk,mk)))

In [19]:
block1 = np.zeros((mk,mk))
a = cp.hstack((sF[0], block1))
bb = cp.hstack((block1,sF[1]))
diag_sF = cp.vstack((a,bb))
block2 = np.zeros((M-mk,mk))
c = cp.hstack((diag_sF,block2))
F_P = cp.vstack((np.zeros((mk,M)),c)) #W-(Psi) in the paper --F_P = Psi * T0_r

In [20]:
sF00 = cp.Variable((mk,nx))
F_0 = cp.vstack((sF00,np.zeros((M-mk,nx)))) # W0 - (Psi_0) --F_0 = Psi_0* T0_r
F_b = cp.Variable((M,nx)) # b - (beta)  还差Psi_f 以及 Beta_f --F_b = beta* T0_r

In [21]:
sPsi = cp.Variable((nf,mk))
Psi_f = cp.hstack((np.zeros((nf,M-mk)),sPsi)) # Psi_f
Beta_f = cp.Variable((nf,1)) # Beta_f 

In [22]:
Gamma_x = cp.Variable((1,1))
gamma = cp.Variable((1,1))

In [23]:
omega = cp.bmat([
    [-Gamma_x*np.eye(nx), W0.T@T0-F_0.T@J1.T, F_0.T-W0.T@Tx, -W0.T@Tc_p, np.zeros((nx,nf))], 
    [(W0.T@T0-F_0.T@J1.T).T, -2*T0+T0@W-W.T@T0.T, (np.eye(N)-W).T@Tx+J1@T0_r-J1@F_P, T0@b+Tp+(np.eye(N)-W).T@Tc_p-J1@F_b, Wf.T],
    [(F_0.T-W0.T@Tx).T, ((np.eye(N)-W).T@Tx+J1@T0_r-J1@F_P).T, -2*T0_r+F_P+F_P.T, F_b+Tp_r-Tx.T@b, -Psi_f.T],
    [(-W0.T@Tc_p).T, (T0@b+Tp+(np.eye(N)-W).T@Tc_p-J1@F_b).T, (F_b+Tp_r-Tx.T@b).T, -Tc_p.T@b-b.T@Tc_p-gamma, bl-Beta_f],
    [(np.zeros((nx,nf))).T, Wf.T.T, -Psi_f.T.T, (bl-Beta_f).T, -np.eye(nf)]
])

In [24]:
weight1 = 1.0
weight2 = 1.0
objective = weight1*Gamma_x + weight2*gamma

In [36]:
constraints = [omega <= 0, Gamma_x >= 0, gamma >= 0]
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve()

inf

In [26]:
prob.status

'infeasible'

In [27]:
for cons in constraints:
  print(cons)

Vstack(Hstack(-var23 * [[1.]], [[ 0.50968695 -0.04696858  0.98947489  0.32785952  0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]] * diag_vec(reshape(var0, (16,))) + -Vstack(var17, [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]).T * [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]], Vstack(var17, [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]).T + -[[ 0.50968695 -0.04696858  0.98947489  0.32785952  0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.  

In [28]:
# # define ReLU
# def ReLU(x):
#     return np.maximum(0,x)

In [29]:
# w0 = model.state_dict()['fc1.weight']
# b0 = model.state_dict()['fc1.bias']
# w1 = model.state_dict()['fc2.weight']
# b1 = model.state_dict()['fc2.bias']
# y1 = torch.matmul(w0, input)+b0
# y1 = F.relu(y1)
# y2 = torch.matmul(w1, y1)+b1
# print(y2)

In [30]:
# y1 = np.matmul(w0, input)+b0
# y1 = ReLU(y1)
# y2 = np.matmul(w1, y1)+b1
# print(y2)